### Chat API : OpenAI
Let's start with a direct API calls to OpenAI.

In [4]:
!python --version

!pip install langchain
!pip install openai
!pip install python-dotenv

Python 3.13.4
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.42-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-win_amd64.whl.metadata (2.1 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.wh

In [16]:
import os
from openai import OpenAI
client = OpenAI()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# Read API key from environment variable
# Ensure you have set the OPENAI_API_KEY in your .env file
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable in your .env file.")
else:
    print("OPENAI_API_KEY is set.")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

print(get_completion("What is 1+1?"))


OPENAI_API_KEY is set.
1 + 1 equals 2.


Function to call the OpenAI LLM

In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


In [ ]:
# Test the function
prompt = "What is 1+1?"
llm_response = get_completion(prompt)
print(f"LLM Response: {llm_response}")

'1 + 1 equals 2.'

Use Case : 
The below email is an example of a customer complaint.
customer is angry and frustrated about the blender lid flying off and making a mess.
The language used is informal and includes pirate slang.

Exact english translation:
I am very angry that the lid of my blender flew off and made a mess in my kitchen with smoothie! And to make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I need your help right now!

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [22]:
output_style = """American English in a calm and respectful tone"""

prompt_to_llm = f"""Translate the text that is delimited by triple backticks into a style that is {output_style}. text: ```{customer_email}```"""

In [ ]:
llm_response_translation = get_completion(prompt_to_llm)
print(f"LLM Response Translation: {llm_response_translation}")

LLM Response: I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would appreciate your assistance with this issue. Thank you.


LLM Response: I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you!

### Chat API : LangChain
Let's try how we can do the same using LangChain.

- chatmodels have been moved to langchain-openai
- Will be deprecated in future versions
- Ensure you have the latest version of langchain-openai

In [39]:
!pip install --upgrade langchain 
!pip install langchain-community 
!pip install -U langchain-openai 

In [ ]:
from langchain_openai import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

response = chat.invoke("What is 1+1?")
print(response.content)

1 + 1 equals 2.


#### Prompt template

In [63]:


# prompt_to_llm = f"""Translate the text that is delimited by triple backticks into a style that is {output_style}. text: ```{customer_email}```"""
# llm_response_translation = get_completion(prompt_to_llm)
# print(f"LLM Response Translation: {llm_response_translation}")
# ^^^^^^^^^^^^^^^^^^^ PREVIOUS CODE - WITHOUT LANGCHAIN ^^^^^^^^^^^^^^^^^^^

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

output_style = """American English in a calm and respectful tone"""

# No formatting needed for the template string. Observe that we did not use f-string here. Compare the following with prompt_to_llm above.
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {output_style}. \
text: ```{customer_email}```
"""

In [65]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

for item in prompt_template:
    print(f"item: {item}")


item: ('name', None)
item: ('input_variables', ['customer_email', 'output_style'])
item: ('optional_variables', [])
item: ('input_types', {})
item: ('output_parser', None)
item: ('partial_variables', {})
item: ('metadata', None)
item: ('tags', None)
item: ('messages', [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['customer_email', 'output_style'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {output_style}. text: ```{customer_email}```\n'), additional_kwargs={})])
item: ('validate_template', False)


In [66]:
# Accessing the messages in the prompt template
print("="*50)
print("Messages in the prompt template:")
for item in prompt_template:
    if item[0] == "messages":
        print(f"item[0]: {item[0]}")
        print(f"item[1]: {item[1]}")

Messages in the prompt template:
item[0]: messages
item[1]: [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['customer_email', 'output_style'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {output_style}. text: ```{customer_email}```\n'), additional_kwargs={})]


In [67]:
print(type(prompt_template))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [68]:
print(prompt_template.messages[0].prompt)

input_variables=['customer_email', 'output_style'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {output_style}. text: ```{customer_email}```\n'


In [69]:
print(prompt_template.messages[0].prompt.input_variables)

['customer_email', 'output_style']


In [71]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

output_style = """American English in a calm and respectful tone"""

# No formatting needed for the template string. Observe that we did not use f-string here. Compare the following with prompt_to_llm above.
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {output_style}. \
text: ```{customer_email}```
"""


customer_messages = prompt_template.format_messages(
    output_style=output_style,
    customer_email=customer_email
)


In [79]:
print(type(customer_messages))
for message in customer_messages:
    print(f"Message >>>>>>>>>>>>>>> {message}")

<class 'list'>
Message >>>>>>>>>>>>>>> content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [80]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [ ]:
# chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# response = chat.invoke("What is 1+1?")
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ PREVIOUS TESTING CODE ^^^^^^^^^^^^^^^^^^^

# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [83]:
print(customer_response.content)

I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you.


In [89]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    output_style=service_style_pirate,
    customer_email=service_reply)

print(service_messages[0].content)


Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [ ]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy, me hearty! I be sorry to inform ye that the warranty be not coverin' the expenses fer cleanin' yer galley, as it seems ye may have misused yer blender by forgettin' to secure the lid afore settin' it to whirl. Aye, 'tis a bit of tough luck, indeed! Fair winds to ye, and may yer future adventures be less messy!
